In [268]:
import pandas as pd
import numpy as np
import io
from bs4 import BeautifulSoup

# To be fixed : doubles codes postaux et valeurs NaN dans income

# Import the data
### candidates
First we create a dataframe  **`candidates`**  with the list of candidates

In [269]:
f = io.open('data/list_candidates.xml', encoding='utf-8')
soup = BeautifulSoup(f, features='xml')
cand_list = soup.find_all('Candidat')
candidates = []

for c in cand_list:
    temp_dict = {}
    temp_dict['cand_id'] = c.find_all('NumPanneauCand')[0].text
    temp_dict['last_name'] = c.find_all('NomPsn')[0].text
    temp_dict['first_name'] = c.find_all('PrenomPsn')[0].text
    temp_dict['civilite'] = c.find_all('CivilitePsn')[0].text
    candidates.append(temp_dict)
f.close()
candidates = pd.DataFrame(candidates)

### election results
We then build the dataframe  **`results`**  containing all the elections results

In [281]:
with io.open('data/results_pres_election.txt', encoding='iso-8859-1') as f:
    line = f.readline()
    
header = ['dpt_code','dpt_name','circo_code','circo_name','city_code','city_name','poll_station',
          'registered','abst','abst_perc','voting','voting_perc','white','white_perc_ins','white_perc_vot',
          'nullv', 'nullv_perc_ins', 'nullv_perc_vot','valid','valid_perc_ins','valid_perc_vot'
         ]

def pattern(i):
    tmp = ['cand_id','sex','last_name','first_name','votes','perc_ins','perc_exp']
    for j in range(len(tmp)):
        tmp[j] = str(i) + '_' + tmp[j]
    return tmp

for i in range(1,len(candidates)+1):
    header.extend(pattern(i))

In [324]:
results = pd.read_csv('data/results_pres_election.txt', sep=';',  encoding='iso-8859-1', header=0, names = header, dtype={'dpt_code': 'str', 'city_code':'str'})

We create a smaller dataframe  **`df`**  with the interesting columns

In [325]:
colums_to_keep = results.columns[[range(0,9) + [10] + [12] + [15] + [18] + range(25,98,7)]]
df = results[colums_to_keep]

### income data

we create a dataframe  **`income`**  with income data by city

In [273]:
income = pd.read_excel('data/income_by_city.xls', skiprows=4)
income_dict = {}
for col in income.columns.values:
    income_dict[income[col][0]] = col
income = pd.read_excel('data/income_by_city.xls', skiprows=5)

In [361]:
income.head(2)

,CODGEO,LIBGEO,NBMENFISC13,NBPERSMENFISC13,MED13,PIMP13,TP6013,TP60AGE113,TP60AGE213,TP60AGE313,...,PPEN13,PPAT13,PPSOC13,PPFAM13,PPMINI13,PPLOGT13,PIMPOT13,RD13,D113,D913
0,01001,L'Abergement-Clémenciat,297.0,768.0,22130.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01002,L'Abergement-de-Varey,99.0,239.0,23213.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
income['MED13'].isnull()

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16        True
17       False
18       False
19        True
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
36556    False
36557    False
36558    False
36559    False
36560    False
36561    False
36562    False
36563    False
36564    False
36565    False
36566    False
36567     True
36568    False
36569    False
36570    False
36571    False
36572     True
36573    False
36574    False
36575    False
36576    False
36577    False
36578    False
36579    False
36580    False
36581    False
36582    False
36583    False
36584    False
36585    False
Name: MED13, Length: 36586, dtype: bool

### table from insee code to zipcode

In [341]:
insee_to_zip = pd.read_csv('data/insee_to_zipcode.csv', sep=';', dtype='str')
insee_to_zip.head(3)

,insee_com,postal_code,nom_comm,nom_dept,nom_region,statut,z_moyen,superficie,population,geo_point_2d,geo_shape,id_geofla,code_comm,code_cant,code_arr,code_dept,code_reg
0,32460,32720,VERGOIGNAN,GERS,MIDI-PYRENEES,Commune simple,126.0,1056.0,0.3,"43.7235746425, -0.188266221507","{""type"": ""Polygon"", ""coordinates"": [[[-0.19884...",34962,460,24,3,32,73
1,51141,51240,LA CHAUSSEE-SUR-MARNE,MARNE,CHAMPAGNE-ARDENNE,Commune simple,130.0,2240.0,0.7,"48.8433156105, 4.54286173009","{""type"": ""Polygon"", ""coordinates"": [[[4.504753...",18178,141,33,4,51,21
2,77130,77580,COULOMMES,SEINE-ET-MARNE,ILE-DE-FRANCE,Commune simple,136.0,371.0,0.4,"48.8919104938, 2.92942534432","{""type"": ""Polygon"", ""coordinates"": [[[2.940606...",31737,130,08,1,77,11


# Cleaning the data

In [326]:
sorted(list(set(df['dpt_code'].values)))[-7:]
results_abroad_idx = df['dpt_code'].isin(sorted(list(set(df['dpt_code'].values)))[-7:])
results_abroad = df[results_abroad_idx]
df = df[~results_abroad_idx]

In [327]:
df['dpt_code'].replace('Z.',u'97', regex=True, inplace=True)
temp = df['dpt_code'] + df['city_code']
df.loc[:,'insee_code'] = temp
df = pd.merge(df, insee_to_zip[['insee_com','postal_code']], how='left', left_on='insee_code', right_on='insee_com')

List of insee codes which map to several zip code (will have to be fixed)

In [360]:
list(set(df[df['postal_code'].str.len() > 5]['postal_code']))

['06130/06520',
 '30000/30900',
 '64122/64700',
 '45000/45100',
 '66000/66100',
 '92190/92360',
 '60113/60190',
 '95000/95300',
 '39310/01410',
 '15000/15250',
 '20251/20270',
 '13080/13090/13100/13290/13540',
 '37000/37100/37200',
 '97200/97234',
 '59140/59240/59430/59640/59279',
 '31000/31100/31200/31300/31400/31500',
 '59000/59160/59260/59777/59800',
 '49000/49100',
 '76600/76610/76620',
 '95000/95800',
 '59491/59493/59650',
 '72000/72100',
 '97139/97142',
 '38000/38100',
 '35000/35200/35700',
 '76000/76100',
 '42000/42100/42230',
 '07310/07320',
 '65200/65710',
 '83700/83530',
 '20200/20600',
 '33000/33100/33200/33300/33800',
 '68630/68126',
 '67000/67100/67200',
 '06600/06160',
 '87000/87100/87280',
 '67130/67570',
 '57000/57050/57070',
 '83600/83370',
 '06400/06150',
 '44000/44100/44200/44300',
 '50100/50130',
 '93200/93210',
 '94100/94210',
 '06000/06100/06200/06300',
 '68100/68200',
 '83000/83100/83200',
 '34000/34070/34080/34090',
 '63000/63100']

In [358]:
df[df['city_code'].str.len() != 3]

,dpt_code,dpt_name,circo_code,circo_name,city_code,city_name,poll_station,registered,abst,voting,...,5_votes,6_votes,7_votes,8_votes,9_votes,10_votes,11_votes,insee_code,insee_com,postal_code


# Groupby test

In [214]:
f = {'dpt_name': lambda x: list(set(x))[0], 
     'registered':['sum'], 'abst':['sum'], 'voting':['sum'], 'white':['sum'], 
     'nullv':['sum'], 'valid':['sum'], '1_votes':['sum'], '2_votes':['sum'], '3_votes':['sum'], 
     '4_votes':['sum'], '5_votes':['sum'], '6_votes':['sum'], '7_votes':['sum'], '8_votes':['sum'], 
     '9_votes':['sum'], '10_votes':['sum'], '11_votes':['sum']}

In [215]:
votes_by_dpt = df.groupby('dpt_code').agg(f)

In [255]:
votes_by_dpt.head(3)

,abst,5_votes,dpt_name,9_votes,7_votes,8_votes,11_votes,registered,6_votes,3_votes,nullv,10_votes,valid,2_votes,white,voting,1_votes,4_votes
,sum,sum,<lambda>,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
dpt_code,,,,,,,,,,,,,,,,,,
01,81507,1842,Ain,51736,595,3465,69805,415886,3098,73692,2238,3612,325799,81455,6342,334379,19788,16711
02,80183,2763,Aisne,48950,536,2264,46969,375752,3156,51680,2382,2171,288140,102770,5047,295569,14651,12230
03,54275,1540,Allier,38311,353,2986,36457,253508,2322,45651,2438,1422,192460,43004,4335,199233,9795,10619
